In [30]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import KFold

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
kfold_n = 5

In [32]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [33]:
def kFold(data_set):
    train_k = []
    valid_k = []
    
    KF = KFold(n_splits = kfold_n)
    for train_index, valid_index in KF.split(data_set):
        train_set, valid_set = data_set[train_index], data_set[valid_index]
        train_k.append(np.array(train_set))
        valid_k.append(np.array(valid_set))
    
    return np.array(train_k), np.array(valid_k)

In [34]:
class CustomDataset(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y_data = y
        else:
            self.y_data = torch.FloatTensor(y)
        self.x_data = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y_data is None:
            return self.x_data[idx]
        else:
            return self.x_data[idx], self.y_data[idx]

    def __len__(self):
        return len(self.x_data)

In [35]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # making the model more elaborative helps reducing the loss
        # trial 1: set Linear(), input_dim -> 16 -> 8 -> 1
        # result: could generate the result, but the loss is not reduced satisfactorilly
        
        # trial 2: 128, and reduce by /2
        # result: at some folds, the train loss could not be decreased,
        #         as well as the validation loss.
        #         they are overfitted, so reduce the to 64
        
        # trial 3: 64 -> 16 -> then reduce by 2
        # result: it produces the most loss at around 1.5 or less.
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

In [36]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

In [37]:
def trainer(train_loader, valid_loader, model, k, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9) 

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = 3000, math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), f'./models/model{k}.ckpt') # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= 400:
            print('\nModel is not improving, so we halt the training session.')
            return

In [38]:
# Set seed for reproducibility
same_seed(5201314)

train_data, test_data = pd.read_csv('../input/ml2022spring-hw1/covid.train.csv').values, pd.read_csv('../input/ml2022spring-hw1/covid.test.csv').values
train_data, valid_data = kFold(train_data)

for k in range(kfold_n):
    # Select features
    x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data[k], valid_data[k], test_data, True)

    train_dataset, valid_dataset, test_dataset = CustomDataset(x_train, y_train), \
                                                CustomDataset(x_valid, y_valid), \
                                                CustomDataset(x_test)

    # Pytorch data loader loads pytorch dataset into batches.
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, pin_memory=True)
    
    print('************************************')
    print(f'{k+1} Fold:')
    print('************************************')
    
    model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
    trainer(train_loader, valid_loader, model, k, device)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


************************************
1 Fold:
************************************


Epoch [1/3000]: 100%|██████████| 9/9 [00:00<00:00, 152.88it/s, loss=111]


Epoch [1/3000]: Train loss: 141.5305, Valid loss: 90.2108
Saving model with loss 90.211...


Epoch [2/3000]: 100%|██████████| 9/9 [00:00<00:00, 154.75it/s, loss=67.9]


...

Epoch [178/3000]: Train loss: 4.7629, Valid loss: 18.2129


Epoch [179/3000]: 100%|██████████| 9/9 [00:00<00:00, 138.93it/s, loss=5.82]


Epoch [179/3000]: Train loss: 4.6061, Valid loss: 17.0193


Epoch [180/3000]:  11%|█         | 1/9 [00:00<00:00, 87.87it/s, loss=4.14]


KeyboardInterrupt: 

In [ ]:
def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

In [ ]:
def save_pred(preds, file):
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])


test_dataset = CustomDataset(test_data, test=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

preds = [0] * (train_dataset.__len__())
for i in range(kfold_n):
    model = My_Model(input_dim=train_dataset.__len__()).to(device)
    model.load_state_dict(torch.load(f'./models/model{i}.ckpt'))
    for j, element in enumerate(predict(test_loader, model, device).tolist()):
        preds[j] += element

for i in range(train_dataset.__len__()):
    # divide by the number of kfold iteration to get average
    preds[i] = preds[i]/kfold_n
    
save_pred(preds, 'pred.csv')